# OECD Producer Price Index

This Python script visualizes the Producer Price Index (PPI) trends of various countries from Jan 2011 to Jan 2023 using dimensionality reduction techniques.

## Data

The [producer price index](https://en.wikipedia.org/wiki/Producer_price_index) (PPI) measures the rate of change of price for products sold as they leave the producer. [OECD](http://oecd.org/), an intergovernmental organization, maintains a dataset of PPI for countries around the world. In this assignment, you will visualize the PPI of various countries from Jan 2011 to Jan 2023 as high dimensional data.

* [PPI dataset](https://data.oecd.org/price/producer-price-indices-ppi.htm#indicator-chart)

The important columns of this dataset are `LOCATION`, `TIME` and `Value`. We will treat the per-country PPI values over time as a single data point. I.e. Each high dimension data point consists of all the values from Jan 2011 to Jan 2023 for a given country. You may want to use `pandas.pivot` to switch the data frame from long form to wide form. For this assignment, we will replace all `NaN` values by 0. 

## Preprocessing:
* The dataset is pivoted so that each country's PPI values over time form a single high-dimensional data point.
* Missing values are replaced with 0, and data is standardized for consistency.

## Dimensionality Reduction:
* PCA (Principal Component Analysis): Reduces high-dimensional data to two principal components.
* MDS (Multidimensional Scaling): Provides an alternative 2D representation based on pairwise distances.

## Visualization:
* Two scatter plots are generated using Altair, where each country is a point colored by location.
* The x and y axes represent the computed components, highlighting clusters of countries with similar PPI trends.
* The final 800x600 resolution charts allow for comparison between PCA and MDS-based embeddings.

In [1]:
!pip install -U scikit-learn

In [2]:
import altair as alt
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import TSNE


url = "https://github.com/qnzhou/practical_data_visualization_in_python/files/14559866/oecd_ppi.csv"
data = pd.read_csv(url)

In [3]:
# Your code here...

data

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUT,PPI,DOMESTIC,IDX2015,M,2011-01,98.65053,NaN
1,AUT,PPI,DOMESTIC,IDX2015,M,2011-02,99.12756,NaN
2,AUT,PPI,DOMESTIC,IDX2015,M,2011-03,99.98622,NaN
3,AUT,PPI,DOMESTIC,IDX2015,M,2011-04,100.36780,NaN
4,AUT,PPI,DOMESTIC,IDX2015,M,2011-05,100.36780,NaN
...,...,...,...,...,...,...,...,...
5746,EU27_2020,PPI,DOMESTIC,IDX2015,M,2022-08,133.80000,NaN
5747,EU27_2020,PPI,DOMESTIC,IDX2015,M,2022-09,134.40000,NaN
5748,EU27_2020,PPI,DOMESTIC,IDX2015,M,2022-10,135.80000,NaN
5749,EU27_2020,PPI,DOMESTIC,IDX2015,M,2022-11,134.90000,NaN


In [4]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
# Pivot the table to get countries as rows and months as columns
pivot_table = data.pivot(index='LOCATION', columns='TIME', values='Value').fillna(0)

pivot_table

TIME,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,...,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01
LOCATION,,,,,,,,,,,,,,,,,,,,,
AUT,98.65053,99.12756,99.98622,100.36780,100.36780,100.46330,100.46330,100.36780,100.46330,100.55870,...,125.3637,126.6903,128.1275,128.0169,127.2430,128.7908,127.9063,126.4692,127.1325,0.0000
BEL,105.10000,106.20000,107.10000,107.90000,107.70000,107.60000,107.60000,107.30000,107.50000,107.40000,...,155.0000,157.3000,160.2000,158.2000,157.7000,156.2000,158.1000,157.1000,155.3000,0.0000
CHE,105.01240,105.27230,105.64670,105.65280,105.47970,105.13020,104.75290,104.40430,104.50530,104.24700,...,107.2716,108.3059,108.9728,109.3016,109.6301,109.7087,109.6758,109.7475,109.4631,0.0000
COL,89.16204,90.01339,90.61604,90.84561,91.39085,91.40999,91.10388,91.38129,91.48651,92.16568,...,143.0170,145.6189,145.6189,149.2730,148.4886,149.8948,152.2862,154.2950,153.5393,0.0000
CRI,84.55310,85.49834,87.08256,88.83438,89.42461,90.18393,90.41148,90.10818,89.99454,90.05784,...,131.6622,134.6859,137.2771,138.3730,137.4205,136.5310,136.1210,135.5293,134.7552,0.0000
CZE,98.00000,98.50000,99.70000,100.60000,101.00000,100.90000,100.80000,100.80000,101.00000,101.10000,...,129.5000,133.6000,136.2000,135.6000,134.4000,134.2000,136.0000,134.5000,132.0000,0.0000
DEU,98.64141,99.41129,99.89247,100.37360,100.37360,100.56610,100.66240,100.46990,100.75860,100.66240,...,126.8000,128.5000,129.1000,129.6000,129.9000,131.0000,131.8000,131.2000,130.7000,0.0000
DNK,95.50000,95.70000,96.40000,97.10000,97.40000,97.60000,97.90000,97.90000,98.40000,98.20000,...,125.4000,126.6000,128.3000,129.7000,129.9000,130.2000,131.3000,130.3000,129.6000,0.0000
EA19,99.50000,100.40000,101.30000,102.00000,101.80000,101.80000,102.10000,101.90000,102.20000,102.00000,...,128.6000,130.6000,132.5000,132.1000,131.6000,132.1000,133.4000,132.6000,131.3000,0.0000


In [6]:
# Standardize the data
scaler = StandardScaler()
pivot_table_scaled = scaler.fit_transform(pivot_table)

**PCA**

In [7]:
# Apply PCA to reduce dimensions to 2D
pca = PCA(n_components=2)
principal_components = pca.fit_transform(pivot_table_scaled)

In [8]:
# Create a DataFrame with the principal components
components_df_PCA = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
components_df_PCA['LOCATION'] = pivot_table.index

In [9]:
# Now, we will create a 2D scatter plot using the two principal components and color each point by its location.
scatter_plot_PCA = alt.Chart(components_df_PCA).mark_point(size=60).encode(
    alt.X('PC1:Q', title='Principal Component 1'),
    alt.Y('PC2:Q', title='Principal Component 2'),
    color='LOCATION:N',
    tooltip=['LOCATION:N', 'PC1:Q', 'PC2:Q']
).properties(
    width=800,
    height=600
)

# Display the chart
scatter_plot_PCA.display()

alt.Chart(...)

**MDS**

In [10]:
# Apply MDS to reduce dimensions to 2D
mds = MDS(n_components=2, random_state=0)
mds_components = mds.fit_transform(pivot_table_scaled)

In [11]:
# Create a DataFrame with the MDS components
components_df_MDS = pd.DataFrame(data=mds_components, columns=['MDS1', 'MDS2'])
components_df_MDS['LOCATION'] = pivot_table.index

In [12]:
# Create a 2D scatter plot using Altair
scatter_plot_MDS = alt.Chart(components_df_MDS).mark_point(size=60).encode(
    alt.X('MDS1:Q', title='MDS Component 1'),
    alt.Y('MDS2:Q', title='MDS Component 2'),
    color='LOCATION:N',
    tooltip=['LOCATION:N', 'MDS1:Q', 'MDS2:Q']
).properties(
    width=800,
    height=600
)

# Display the chart
scatter_plot_MDS.display()

alt.Chart(...)